In [1]:
import itertools
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
%run keras_sentinel.ipynb

2024-05-18 10:57:30.274101: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 10:57:31.135593: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

shuffle_seed = 42
shuffled_labels = shuffle(labels, random_state=shuffle_seed)

test_years = [2020]
test_seasons = [6]

train_years = [2017, 2018, 2019]
all_seasons = [3, 6, 9, 12]

season_combinations = itertools.chain.from_iterable(
    itertools.combinations(all_seasons, r) for r in range(1, len(all_seasons)+1))
season_combinations = list(season_combinations)

In [ ]:
results = []
for season_combination in tqdm(season_combinations[-1:]):
    filepaths = []
    for year in train_years:
        for season in season_combination:
            file_key = f'processed_treesat_{year}*{season}.npy'
            filepaths.extend(sorted(list(data_dir.glob(file_key))))
            
    selected_data = []
    for filepath in filepaths:
        with open(filepath, 'rb') as f:
            data = np.load(f)
        selected_data.append(shuffle(data, random_state=shuffle_seed))
    features = np.stack(selected_data, axis=1)

    model_name = f'conv_seasons_{"_".join(map(str, season_combination))}_years_{"_".join(map(str, train_years))}.keras'
    model_path = model_dir.joinpath(model_name)
    model_eval = KerasModelCreator().run(
        shuffled_labels, features, model_path, epochs=3, overwrite=True)
    results.append(model_eval)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/3


2024-05-18 10:57:58.077310: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1018562688 exceeds 10% of free system memory.


45342/45342 ━━━━━━━━━━━━━━━━━━━━ 1455s 32ms/step - accuracy: 0.3490 - f1_majority: 0.1411 - loss: 0.0217 - r2_score: 0.0732 - root_mean_squared_error: 0.2081
Epoch 2/3
45342/45342 ━━━━━━━━━━━━━━━━━━━━ 1443s 32ms/step - accuracy: 0.5392 - f1_majority: 0.3723 - loss: 0.0164 - r2_score: 0.2190 - root_mean_squared_error: 0.1809
Epoch 3/3
 7652/45342 ━━━━━━━━━━━━━━━━━━━━ 19:58 32ms/step - accuracy: 0.5841 - f1_majority: 0.4359 - loss: 0.0151 - r2_score: nan - root_mean_squared_error: 0.1738

In [ ]:
# import subprocess
# subprocess.run('sudo', 'shutdown', 'now')

In [ ]:
# results_df = pd.DataFrame(results)
# results_df.insert(0, 'season_comb', season_combinations)
# results_df

In [ ]:
results = []
for season_combination in tqdm(season_combinations[-1:]):
    selected_data = []
    for test_year in test_years:
        for season in season_combination:
            filepath = data_dir.joinpath(f'processed_treesat_{test_year}{str(season).zfill(2)}.npy')
            with open(filepath, 'rb') as f:
                data = np.load(f)
            selected_data.append(shuffle(data, random_state=shuffle_seed))
    features = np.stack(selected_data, axis=1)
    
    y = KerasModelCreator().crop_y(shuffled_labels)
    for i in range(features.shape[-1]):
        features[...,i] = KerasModelCreator().normalise_X(features[...,i])
    
    model_name = f'conv_seasons_{"_".join(map(str, season_combination))}_years_{"_".join(map(str, train_years))}.keras'
    model = tf.keras.models.load_model(model_dir.joinpath(model_name))

    model_eval = model.evaluate(
        features, y, verbose=0, return_dict=True
    )
    
    results.append(model_eval)

In [ ]:
results

In [ ]:
results = []
for season in test_seasons:
    selected_data = []
    for year in train_years:
        file_key = f'processed_treesat_{test_year}{str(season).zfill(2)}.npy'
        filepath = data_dir.joinpath(file_key)
        with open(filepath, 'rb') as f:
            data = np.load(f)
        selected_data.append(data)
        
    features = np.concatenate(selected_data, axis=0)
    matching_labels = np.array([labels]*features.shape[0])

    model_name = f'conv_seasons_{"_".join(map(str, test_seasons))}_years_{"_".join(map(str, train_years))}.keras'
    model_path = model_dir.joinpath(model_name)
    model_eval = KerasModelCreator().run(
        matching_labels, features, model_path, epochs=3, overwrite=True)
    results.append(model_eval)

In [ ]:
# major_pred = y_pred.argmax(axis=1)
# major_true = y_true.argmax(axis=1)

# cr = classification_report(
#     major_true, major_pred, target_names=df[target].columns, 
#     output_dict=True, zero_division=np.nan
# )
# df_report = pd.DataFrame(cr).T
# df_report['support'] = df_report['support'].astype(int)

# display(HTML(df_report.round(decimals=2).to_html()))